In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, Normalizer
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from mbspbs10pc import mbs_online
reload(mbs_online);

nltk.download('stopwords')

/home/samu/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /home/samu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load data

In [2]:
labels = pd.read_csv('../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv', header=0).rename({'Unnamed: 0': 'PIN'}, axis=1)[['PIN', 'CLASS']].set_index('PIN')
data = pd.read_csv('../../tmp/raw_sequences.csv', header=0, index_col=0).loc[labels.index, 'seq']

df = pd.DataFrame(columns=['Seq', 'Class', 'mbs_seq', 'times_seq'], index=data.index)

df.loc[:, 'Seq'] = data
df.loc[:, 'Class'] = labels['CLASS']

for idx in df.index:
    _tmp = df.loc[idx, 'Seq'].split(' ')
    df.loc[idx, 'mbs_seq'] = ' '.join(_tmp[::2])
    df.loc[idx, 'times_seq'] = ' '.join(_tmp[1::2])

In [3]:
df.head()

,Seq,Class,mbs_seq,times_seq
PIN,,,,
568483,10990 0 23 1 725 0 10990 0 10990 0 10990 0 109...,1,10990 23 725 10990 10990 10990 10993 23 85311 ...,0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 ...
923748,66653 0 65070 0 66515 0 10962 0 105 0 56807 2 ...,0,66653 65070 66515 10962 105 56807 10990 23 109...,0 0 0 0 0 2 0 1 2 0 0 0 0 0 0 0 0 0 0 0 1 2 1 ...
2035328,66536 0 74995 0 66512 0 66551 0 66608 0 66716 ...,0,66536 74995 66512 66551 66608 66716 66560 7392...,0 0 0 0 0 0 0 0 0 0 0 3 0 0 2 1 2 0 0 0 0 0 0 ...
2239386,10991 0 23 3 73928 0 66560 0 66515 0 66551 0 6...,1,10991 23 73928 66560 66515 66551 66536 2517 10...,0 3 0 0 0 0 0 0 1 0 0 0 0 1 3 0 0 0 0 0 0 2 0 ...
2244533,10990 0 23 2 23 0 10990 0 73928 0 66500 0 6656...,1,10990 23 23 10990 73928 66500 66560 66551 23 1...,0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


# Seq2token

In [4]:
# Define tokenizer object
seq_tokenizer = Tokenizer(char_level=False, lower=False, split=' ')

# Fit on corpus and extract tokenized sequences
seq_tokenizer.fit_on_texts(df['mbs_seq'])
mbs_tokens = map(int, seq_tokenizer.word_index.keys())
print(len(mbs_tokens))
print(mbs_tokens[:3])

2773
[65096, 65093, 65090]


# Get tokens from `item_map.tsv`

In [5]:
item_map = pd.read_csv('../mbspbs10pc/data/item_map.tsv', sep='\t', header=0, index_col=0)
item_map.head()

IOError: File ../mbspbs10pc/data/item_map.tsv does not exist

In [ ]:
df = item_map.loc[mbs_tokens, 'Mapped_Item_Desc'].to_frame().dropna()
print(df.head())
print(df.shape)

# Description preprocessing

## Removing stop words and punctuation

In [ ]:
regexp_tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
cleaned_mapped_items_desc = {}
for idx in df.index:
    word_list = regexp_tokenizer.tokenize(df.loc[idx].values[0])  # tokenize
    word_list = filter(lambda x: len(x)>3, word_list)  # remove short words 
    cleaned_mapped_items_desc[idx] = [word.lower() for word in word_list if word not in stopwords.words('english')]

## Parsing the GloVe word-embeddings file

In [ ]:
glove_dir = '../glove6B/'
D = 50
# D = 100
# D = 200
# D = 300
with open(os.path.join(glove_dir, 'glove.6B.{}d.txt'.format(D))) as f:
    embeddings_index = {}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

## Prepare the embedding matrix

In [ ]:
# this copes with descriptions like: "3 or more tests described in item 73802"

embedding = {}
for mbs_item in cleaned_mapped_items_desc.keys(): # for each mbs item
    word_list = list(cleaned_mapped_items_desc[mbs_item]) # get the clean word list (copy)
    tail = []  # buffer where to save all the tails
    for word in word_list:
        if word.isdigit() and int(word) in cleaned_mapped_items_desc:
            tail.extend(cleaned_mapped_items_desc[int(word)])
    word_list.extend(tail) # collapse all word in a single list
    
    embedding_vector = [] # and create the mebedding
    for word in word_list:
        if word in embeddings_index:
            embedding_vector.append(embeddings_index[word])
    if len(embedding_vector) == 0:
        print('Dummy encoding for {}'.format(mbs_item))
        embedding[mbs_item] = np.random.randn(D)
    else:
        embedding_vector = np.array(embedding_vector)
        embedding[mbs_item] = np.mean(embedding_vector, axis=0) # average up all the vectors        
embedding = pd.DataFrame.from_dict(embedding, orient='index')
embedding = pd.DataFrame(Normalizer().fit_transform(embedding.values), # then finally normalize everything
                         index=embedding.index)
embedding.to_csv('../../tmp/embedding.{}d.csv'.format(D))

# MBS embedding viz

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
# dimred = TSNE(n_components=3)
# embedding_r = dimred.fit_transform(embedding.values)

In [ ]:
# fig = plt.figure(dpi=500)
# ax = fig.add_subplot(111, projection='3d')

# # idx = np.random.randint(embedding_r.shape[0], size=(60,))
# xx, yy, zz = embedding_r[idx, 0], embedding_r[idx, 1], embedding_r[idx, 2]

# ax.scatter(xx, yy, zz)

# for x, y, z, txt in zip(xx, yy, zz, embedding.index):
#     ax.text(x, y, z, str(txt), zdir='y', fontsize=4)

# ax.set_xlabel('x1')
# ax.set_ylabel('x2')
# ax.set_zlabel('x3');